In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as matdates
from datetime import datetime as dt
from datetime import date, timedelta
import time
import yfinance as yf
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
stock_list = {'SPY' : '미국 대형주/SPY',
              'EFA' : '미국 외 선진국 주식/EFA',
              'BIL' : '초단기채권/BIL'} 

In [4]:
def to_index(x): # 인덱스화
    for i in range(1,len(x)):
        x.iloc[i] = (x.iloc[i]/x.iloc[0])*100
    x.iloc[0] = 100
def get_return(x):  #기간 동안의 첫번째 종가와 마지막 시가로 수익률 계산
    global stock_return
    rate_of_return = round((x.iloc[-1]-100),2)
    stock_return.append(rate_of_return)

In [5]:
start_time = time.time()

df = pd.DataFrame({'12 Month' : []})
end = dt.now()
start = end - timedelta(weeks=48)
stock_return = []
stock_category = []
ten_mon_avg = []
current_value = []

for code, category in zip(stock_list.keys(), stock_list.values()):
    current = yf.download(code, end - timedelta(days=3))['Close']
    current_value.append(current[-1])
    code = yf.download(code, start)['Close']
    to_index(code)
    get_return(code)
    stock_category.append(category)
df['12 Month'] = stock_return
df['Category'] = stock_category
ctgrset = df['Category'].copy().str.split('/', expand=True)
df['Category'] = ctgrset[0]
df['Ticker'] = ctgrset[1]
df = df.sort_values('12 Month', ascending=False).reset_index(drop=True)
df.insert(1, 'Top Rank', range(1, 1 + len(df)))

print("--- %s seconds ---" % (time.time() - start_time))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
--- 1.6153616905212402 seconds ---


In [6]:
df

,12 Month,Top Rank,Category,Ticker
0,16.94,1,미국 외 선진국 주식,EFA
1,10.04,2,미국 대형주,SPY
2,0.05,3,초단기채권,BIL


In [7]:
# 매월 말 SPY, EFA, BIL(초단기채권)의 최근 12개월 수익률을 계산
# SPY의 수익률이 BIL보다 높으면 SPY 또는 EFA 중 최근 12개월 수익률이 더 높은 ETF에 투자
# SPY의 수익률이 BIL보다 낮으면 AGG에 투자